In [106]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django.conf import settings
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta
import time
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
# trans = m.Transaction.objects.select_related('sender','receiver'); trans
trans = m.Transaction.objects.select_related(
    'sender',
    'sender__boo','sender__boo__user',
    'sender__raffle','sender__raffle__item',
    'sender__shoptem','sender__shoptem__item',
    'sender__coffeecoupon','sender__coffeecoupon__item',
    'sender__support','sender__support__brand',
    'receiver',
    'receiver__boo','receiver__boo__user',
    'receiver__raffle','receiver__raffle__item',
    'receiver__support','receiver__support__brand',
    'receiver__shoptem','receiver__shoptem__item',
    'receiver__coffeecoupon','receiver__coffeecoupon__item',
); trans

<QuerySet [<Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > STARBUCKS LAMP BK - 0 | 2021-08-07 20:37:53>, <Transaction: quantlab | quantlab@kakao.com > TAW&TOE ZEROVITY™ Flip Flop, SIZE M - 38000 | 2021-08-07 23:34:16>, <Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > JOHNBER | 존버 스포츠 | 2021-08-10 18:02:20>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>,

# 모든 트랜젝션 가져오기

In [3]:
trans_list = []
for _t in trans:
    tr = {}
    tr_type = _t.get_type_display()
    tr['user'] = str(_t.sender) if tr_type in ['래플','후원'] else str(_t.receiver)
    tr['when'] = str(_t.when.date())
    tr['type'] = tr_type
    tr['amount'] = -_t.amount if tr_type in ['래플','후원'] else _t.amount
    trans_list.append(tr)

# 트랜젝션 횟수 시계열

In [6]:
counts = pd.DataFrame(trans_list).groupby(['user','type','when']).count().unstack().fillna(0)
counts = counts.groupby('type').sum()[counts.columns[-29:-1]].drop(['첫방문환영','쇼핑','커피쿠폰','테스트 INFLOW']).astype('int64')
#counts.style.set_table_attributes('style="font-size: 10px"')
counts.T

type               관심스타일입력  관심아이템입력  기본정보입력  래플  리서치참여  매일밸런스게임  옷장넣기  출첵게임  \
       when                                                                   
amount 2021-09-13        0        0       3   0     32      402     0    30   
       2021-09-14        1        0       0   0      6      243     0    18   
       2021-09-15        0        0       2   3     23      499     0    19   
       2021-09-16        1        1       3   2     80      712     2    27   
       2021-09-17        0        0       2  12     18     1000     3    37   
       2021-09-18        1        1      10   7     58      639     4    21   
       2021-09-19        1        1       1  13     30      751     6    15   
       2021-09-20        0        0       0   0      8      819     1    16   
       2021-09-21        0        0       2   1     33      775     1    24   
       2021-09-22        0        0       3   2     19      886     3    25   
       2021-09-23        0        0       2   1     28     1041     4    28   
       2021-09-24        2        2       4   5     42      988     2    21   
       2021-09-25        0        0       2   4     37     1167     2    26   
       2021-09-26        2        2       4   4     41     1118     0    32   
       2021-09-27        2        1       4   2     36      853     1    20   
       2021-09-28        1        1       3   3     40      918     3    41   
       2021-09-29        1        1       3   1     42     1080     2    33   
       2021-09-30        2        2       7   3    119     2048     1    52   
       2021-10-01        1        1       6   9     70     1696     0    61   
       2021-10-02        0        0       1   5     16     1683     1    31   
       2021-10-03        1        1       0   2     23     1746     2    25   
       2021-10-04        0        0       0   1     19     1616     5    30   
       2021-10-05        3        3       5   0     16     1809     0    34   
       2021-10-06        0        0       1   8     28     1572     3    39   
       2021-10-07        1        1       1   7     27     1602     0    28   
       2021-10-08        0        0       1  12      7     1624     4    34   
       2021-10-09        1        1       1   6     21     1415     0    41   
       2021-10-10        0        0       2  14     13     1847     1    34   

type               후원  
       when            
amount 2021-09-13   0  
       2021-09-14   0  
       2021-09-15   5  
       2021-09-16   6  
       2021-09-17   3  
       2021-09-18   0  
       2021-09-19   0  
       2021-09-20   0  
       2021-09-21   0  
       2021-09-22   0  
       2021-09-23   0  
       2021-09-24   0  
       2021-09-25   1  
       2021-09-26   0  
       2021-09-27   0  
       2021-09-28   1  
       2021-09-29   1  
       2021-09-30   0  
       2021-10-01   0  
       2021-10-02   0  
       2021-10-03   0  
       2021-10-04   0  
       2021-10-05   0  
       2021-10-06   0  
       2021-10-07   0  
       2021-10-08   0  
       2021-10-09   0  
       2021-10-10   0

# 트랜젝션 금액 시계열

In [7]:
amounts = pd.DataFrame(trans_list).groupby(['user','type','when']).sum().unstack().fillna(0)
amounts = amounts.groupby('type').sum()[amounts.columns[-29:-1]].drop(['첫방문환영','쇼핑','커피쿠폰','테스트 INFLOW']).astype('int64')
#amounts.style.set_table_attributes('style="font-size: 10px"')
amounts.T

type               관심스타일입력  관심아이템입력  기본정보입력    래플  리서치참여  매일밸런스게임  옷장넣기  출첵게임  \
       when                                                                     
amount 2021-09-13        0        0     300     0   3640      400     0   240   
       2021-09-14       50        0       0     0    180      243     0   140   
       2021-09-15        0        0     200  -288   2020      475     0   130   
       2021-09-16       50       50     300  -143   8010      700     2   230   
       2021-09-17        0        0     200 -1050   2140      900     3   230   
       2021-09-18       50       50     400  -641   6270      635     4   190   
       2021-09-19       50       50     100 -1398   2770      685     6   110   
       2021-09-20        0        0       0     0    510      801     1   140   
       2021-09-21        0        0     200   -77   3670      735     1   180   
       2021-09-22        0        0     300  -222   1380      846     3   210   
       2021-09-23        0        0     200   -77   2980      973     4   200   
       2021-09-24      100      100     400  -464   4440      976     2   170   
       2021-09-25        0        0     200  -398   3770     1135     2   200   
       2021-09-26      100      100     400  -410   3890     1098     0   240   
       2021-09-27      100       50     400  -210   3170      831     1   200   
       2021-09-28       50       50     300  -288   4550      808     3   310   
       2021-09-29       50       50     300   -77   4790     1060     2   290   
       2021-09-30      100      100     700  -265  13060     1662     1   460   
       2021-10-01       50       50     600  -875   6640     1442     0   490   
       2021-10-02        0        0     100  -500   1460     1361     1   250   
       2021-10-03       50       50       0  -222   2500     1434     2   230   
       2021-10-04        0        0       0  -111   2420     1364     5   260   
       2021-10-05      150      150     500     0   1460     1649     0   340   
       2021-10-06        0        0     100  -811   3520     1472     3   330   
       2021-10-07       50       50     100  -733   2370     1584     0   260   
       2021-10-08        0        0     100 -1321    480     1508     4   300   
       2021-10-09       50       50     100  -666   2460     1367     0   310   
       2021-10-10        0        0       0 -1532   2550     1685     1   300   

type                후원  
       when             
amount 2021-09-13    0  
       2021-09-14    0  
       2021-09-15 -500  
       2021-09-16 -600  
       2021-09-17 -300  
       2021-09-18    0  
       2021-09-19    0  
       2021-09-20    0  
       2021-09-21    0  
       2021-09-22    0  
       2021-09-23    0  
       2021-09-24    0  
       2021-09-25 -100  
       2021-09-26    0  
       2021-09-27    0  
       2021-09-28 -100  
       2021-09-29 -100  
       2021-09-30    0  
       2021-10-01    0  
       2021-10-02    0  
       2021-10-03    0  
       2021-10-04    0  
       2021-10-05    0  
       2021-10-06    0  
       2021-10-07    0  
       2021-10-08    0  
       2021-10-09    0  
       2021-10-10    0

# 가입자수 추이

In [4]:
qrs = m.User.objects.filter(date_joined__gte='2021-09-13').values('date_joined__date').annotate(count=Count('date_joined__date'))

In [5]:
pd.DataFrame(qrs).set_index('date_joined__date').sort_index(ascending=True).iloc[:-1].sum()

count    238
dtype: int64

# 서베이별 참여자수

In [2]:
_boos = m.Boo.objects.filter(answers__has_key='12')

In [3]:
_boos[10].answers

{'4': {'11': [1],
  '12': [0],
  '13': [1],
  '14': [0],
  '15': [1],
  '16': [1],
  '17': [0],
  '18': [0],
  '19': [0],
  '20': [1],
  '21': [1],
  '22': [0],
  '23': [0],
  '24': [0],
  '25': [0],
  'finished': True},
 '5': {'26': [1],
  '27': [1],
  '28': [1],
  '29': [0],
  '30': [0],
  '31': [1],
  '32': [0, 1],
  '33': [0],
  '34': [1],
  '35': [1],
  '36': [1],
  '37': [0],
  '38': [0],
  '39': [0],
  '40': [0],
  '41': [1],
  'finished': True},
 '6': {'42': [1],
  '43': [0],
  '44': [3],
  '45': [0],
  '46': [0],
  '115': [1],
  '117': [1],
  '118': [1],
  '119': [2],
  '120': [1],
  '121': [5],
  '122': [1, 2],
  '123': [0],
  '124': [1],
  'finished': True},
 '7': {'47': [1],
  '48': [1],
  '49': [0],
  '50': [0],
  '51': [1],
  '171': [0],
  '173': [0],
  '174': [0],
  '175': [1],
  '177': [0],
  '178': [1],
  '179': [0],
  '181': [1],
  '182': [1],
  '183': [0],
  'finished': True},
 '8': {'52': [1],
  '53': [1],
  '54': [1],
  '55': [1],
  '56': [1],
  '58': [1],
  '59': 

In [ ]:
배드보이 4
터치그라운드 5
존버 6
포스토 7
브라스보트 8

비오는날 취향분석 9
슬기로운 캠필생활 10
나에게 딱 맞는 주말 베스트 코디 12
내 휴가 스타일 14
핫서머에 입기 좋은 핫팬츠 취향조사 15
구두취향 본격탐구 17
내 백 취향 18
요즘 막 당신의 관심사 19
비오는날 샌들 취향조사 20
어느 브랜드 좋아해 22
올가을 트랜드 조사 신발편 23
가을 유니폼 후디 맨투맨 고르기 24
부츠 이렇게 신어야지 25
이번 fw 뭐 신지 26


In [4]:
m.Boo.objects.filter(answers__has_key='4').count()

120

In [5]:
pd.DataFrame({
    '배드보이': [m.Boo.objects.filter(answers__has_key='4').count()],
    '터치그라운드': [m.Boo.objects.filter(answers__has_key='5').count()],
    '존버': [m.Boo.objects.filter(answers__has_key='6').count()],
    '포스토': [m.Boo.objects.filter(answers__has_key='7').count()],
    '브라스보트': [m.Boo.objects.filter(answers__has_key='8').count()],
    
    '비오는날 취향분석': [m.Boo.objects.filter(answers__has_key='9').count()],
    '슬기로운 캠핑생활': [m.Boo.objects.filter(answers__has_key='10').count()],
    '주말 베스트 코디': [m.Boo.objects.filter(answers__has_key='12').count()],
    '내 휴가 스타일': [m.Boo.objects.filter(answers__has_key='14').count()],
    '핫팬츠 취향조사': [m.Boo.objects.filter(answers__has_key='15').count()],
    '구두취향 본격탐구': [m.Boo.objects.filter(answers__has_key='17').count()],
    '내 백 취향': [m.Boo.objects.filter(answers__has_key='18').count()],
    '요즘 막 당신의 관심사': [m.Boo.objects.filter(answers__has_key='19').count()],
    '샌들 취향조사': [m.Boo.objects.filter(answers__has_key='20').count()],
    '어느 브랜드 좋아해': [m.Boo.objects.filter(answers__has_key='22').count()],
    '올가을 트랜드 조사 신발편': [m.Boo.objects.filter(answers__has_key='23').count()],
    '가을 유니폼 후디 맨투맨 고르기': [m.Boo.objects.filter(answers__has_key='24').count()],
    '부츠 이렇게 신어야지': [m.Boo.objects.filter(answers__has_key='25').count()]
}).T

,0
배드보이,120
터치그라운드,80
존버,68
포스토,107
브라스보트,82
비오는날 취향분석,84
슬기로운 캠핑생활,79
주말 베스트 코디,87
내 휴가 스타일,43
핫팬츠 취향조사,16


In [17]:
answers = m.Boo.objects.filter(answers__has_key='4').values('id','nick','answers'); answers

<QuerySet [{'id': 105, 'nick': 'leesuyong', 'answers': {'4': {'11': [0], '12': [0], '13': [1], '14': [0], '15': [1], '16': [0], '17': [5, 7, 0, 9], '18': [1], '19': [0, 1, 3, 2], '20': [0], '21': [1], '22': [0], '23': [0], '24': [0], '25': [1], 'finished': True}, '5': {'26': [1], '27': [0], '28': [1], '29': [0], '30': [0], '31': [1], '32': [1, 4], '33': [0], '34': [1], '35': [1], '36': [1], '37': [1], '38': [1], '39': [0], '40': [4, 0, 5], '41': [1], 'finished': True}, '6': {'42': [1], '43': [0], '44': [3, 0, 2], '45': [0], '46': [1], '115': [1], '116': ['살로몬'], '117': [0], '118': [1], '119': [2, 4], '120': [1], '121': [5, 4, 0], '122': [1, 4], '123': [1], '124': [0], 'finished': True}, '7': {'47': [1], '48': [1], '49': [1], '50': [0], '51': ['ZARA 티셔츠'], '171': [0], '173': [0], '174': [0], '175': [1], '177': [0], '178': [0], '179': [1], '181': [1], '182': [0], '183': [0], 'finished': True}, '8': {'52': [1], '53': [1], '54': [0], '55': [0], '56': [0], '57': ['보테가베네타'], '58': [0], '59':

In [21]:
answers_4 = [{'id':o['id'], 'nick':o['nick'], 'answers':o['answers']['4']} for o in answers]; answers_4

[{'id': 105,
  'nick': 'leesuyong',
  'answers': {'11': [0],
   '12': [0],
   '13': [1],
   '14': [0],
   '15': [1],
   '16': [0],
   '17': [5, 7, 0, 9],
   '18': [1],
   '19': [0, 1, 3, 2],
   '20': [0],
   '21': [1],
   '22': [0],
   '23': [0],
   '24': [0],
   '25': [1],
   'finished': True}},
 {'id': 697,
  'nick': 'rainycoke',
  'answers': {'11': [1],
   '12': [0],
   '13': [1],
   '14': [0],
   '15': [1],
   '16': [0],
   'finished': False}},
 {'id': 765,
  'nick': 'parkhd0125',
  'answers': {'11': [1],
   '12': [0],
   '13': [1],
   '14': [0],
   '15': [0],
   '16': [0],
   '17': [0, 5],
   '18': [1],
   '19': [0, 2],
   '20': [0],
   '21': [1],
   '22': [0, 3],
   '23': [0],
   '24': [1, 0],
   '25': [1],
   'finished': True}},
 {'id': 902,
  'nick': '013600',
  'answers': {'11': [1],
   '12': [0],
   '13': [0],
   '14': [1],
   '15': [0],
   '16': [1],
   '17': [10, 2, 5],
   '18': [0],
   '19': [2, 1],
   '20': [1],
   'finished': False}},
 {'id': 615,
  'nick': 'hoo_u',
  'a

In [48]:
df = pd.DataFrame(answers_4); df

,id,nick,answers
0,105,leesuyong,"{'11': [0], '12': [0], '13': [1], '14': [0], '..."
1,697,rainycoke,"{'11': [1], '12': [0], '13': [1], '14': [0], '..."
2,765,parkhd0125,"{'11': [1], '12': [0], '13': [1], '14': [0], '..."
3,902,013600,"{'11': [1], '12': [0], '13': [0], '14': [1], '..."
4,615,hoo_u,"{'11': [1], '12': [0], '13': [1], '14': [1], '..."
...,...,...,...
115,989,theloki7,"{'11': [0], '12': [1], '13': [1], '14': [0], '..."
116,1019,dosungv,"{'11': [0], '12': [0], '13': [0], '14': [0], '..."
117,956,hey2208,"{'11': [0], '12': [0], '13': [1], '14': [0], '..."
118,832,skyehdgnl,"{'11': [1], '12': [1], '13': [1], '14': [1], '..."


In [54]:
def conv(x):
    try:
        if type(x)==bool:
            return x
        
        str_x = [str(n) for n in x]
        return ', '.join(str_x)
    
    except:
        return ''
    
df2 = pd.DataFrame(list(df['answers'].values)).applymap(conv); df2

,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,finished
0,0,0,1,0,1,0,"5, 7, 0, 9",1,"0, 1, 3, 2",0,1,0,0,0,1,True
1,1,0,1,0,1,0,,,,,,,,,,False
2,1,0,1,0,0,0,"0, 5",1,"0, 2",0,1,"0, 3",0,"1, 0",1,True
3,1,0,0,1,0,1,"10, 2, 5",0,"2, 1",1,,,,,,False
4,1,0,1,1,0,1,0,0,0,0,1,5,0,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,1,1,0,1,1,1,0,0,0,1,0,1,0,0,True
116,0,0,0,0,1,1,"0, 1, 2, 9, 5",,,,,,,,,False
117,0,0,1,0,1,1,4,0,1,1,0,4,1,1,0,True
118,1,1,1,1,1,1,"1, 0, 3, 2, 5, 8, 11, 9, 6, 4, 10",0,"2, 0, 5, 3",1,1,"1, 3, 0, 4, 5, 2",0,"0, 1",1,True


In [58]:
df3 = pd.concat([df, df2], axis=1).drop(['answers'], axis=1); df3

,id,nick,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,finished
0,105,leesuyong,0,0,1,0,1,0,"5, 7, 0, 9",1,"0, 1, 3, 2",0,1,0,0,0,1,True
1,697,rainycoke,1,0,1,0,1,0,,,,,,,,,,False
2,765,parkhd0125,1,0,1,0,0,0,"0, 5",1,"0, 2",0,1,"0, 3",0,"1, 0",1,True
3,902,013600,1,0,0,1,0,1,"10, 2, 5",0,"2, 1",1,,,,,,False
4,615,hoo_u,1,0,1,1,0,1,0,0,0,0,1,5,0,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,989,theloki7,0,1,1,0,1,1,1,0,0,0,1,0,1,0,0,True
116,1019,dosungv,0,0,0,0,1,1,"0, 1, 2, 9, 5",,,,,,,,,False
117,956,hey2208,0,0,1,0,1,1,4,0,1,1,0,4,1,1,0,True
118,832,skyehdgnl,1,1,1,1,1,1,"1, 0, 3, 2, 5, 8, 11, 9, 6, 4, 10",0,"2, 0, 5, 3",1,1,"1, 3, 0, 4, 5, 2",0,"0, 1",1,True


In [102]:
df3.to_excel('badboy_result.xlsx')

In [103]:
df4 = pd.DataFrame(m.ResearchItem.objects.filter(pk__in=range(11,26)).values('id','type','text')); df4

,id,type,text
0,11,OX,이건 마스터마인드 브랜드의 로고다!! - O\r\n아니다!! 다른 브랜드의 로고다!...
1,15,OX,나는 한달에 패션 쇼핑을 30만원 이상 하는 편이다!!
2,12,AB,다음 로고와 관련이 깊은 브랜드는??
3,18,AB,배드보이 로고는 이게 낫지!!
4,13,AB,이번 주말에 어디서 데이트 할까??
5,16,OX,브랜드 배드보이를 알고 있다!!
6,14,AB,둘 중 내가 더 좋아하는 스타일은??
7,20,AB,배드보이와 느낌이 비슷한 브랜드는??
8,21,OX,"이 로고, 호감 vs 비호감??"
9,24,MC,이 제품은 얼마가 적당할까?


In [104]:
df4.to_excel('badboy_researchitems.xlsx')

# 포인트 현황

In [111]:
memb_list = ['quantlab@kakao.com',
'gem763@gmail.com',
'sejin76@nate.com',
'magrittelim@gmail.com',
'magrittelim@naver.com',
'smilekhm@gmail.com',
'hmkoo@kakao.com',
'giseok1929@daum.net',
'gentlebaek89@gmail.com',
'leesuyong@hotmail.com']

In [125]:
wallet_stat = []
for w in tqdm(m.Wallet.objects.all()[:]):
    if w.whose_type == 'boo':
        wallet_stat.append({
            'whose': str(w.whose), 
            'n_transaction': w.n_transaction, 
            'inflow': w.inflow, 
            'outflow': w.outflow, 
            'amount': w.amount
        })

  0%|          | 0/459 [00:00<?, ?it/s]

In [126]:
wallet_stat

[{'whose': 'quantlab | quantlab@kakao.com',
  'n_transaction': 917,
  'inflow': 54107,
  'outflow': 10785,
  'amount': 43322},
 {'whose': '모이버 | moiber@moiber.com',
  'n_transaction': 66021,
  'inflow': 1,
  'outflow': 600601,
  'amount': -600600},
 {'whose': '홈마 | quantlab@kakao.com',
  'n_transaction': 0,
  'inflow': 0,
  'outflow': 0,
  'amount': 0},
 {'whose': 'Seoul_domestic_m | magrittelim@gmail.com',
  'n_transaction': 1719,
  'inflow': 105128,
  'outflow': 23710,
  'amount': 81418},
 {'whose': 'minimal_ootd | magrittelim@gmail.com',
  'n_transaction': 0,
  'inflow': 0,
  'outflow': 0,
  'amount': 0},
 {'whose': 'giseok1929 | giseok1929@daum.net',
  'n_transaction': 387,
  'inflow': 53682,
  'outflow': 2060,
  'amount': 51622},
 {'whose': 'gentlebaek89 | gentlebaek89@gmail.com',
  'n_transaction': 1595,
  'inflow': 54829,
  'outflow': 0,
  'amount': 54829},
 {'whose': 'smilekhm | smilekhm@gmail.com',
  'n_transaction': 235,
  'inflow': 52366,
  'outflow': 5165,
  'amount': 47201

In [127]:
df = pd.DataFrame(wallet_stat).sort_values(by=['amount'], ascending=False); df

,whose,n_transaction,inflow,outflow,amount
3,Seoul_domestic_m | magrittelim@gmail.com,1719,105128,23710,81418
6,gentlebaek89 | gentlebaek89@gmail.com,1595,54829,0,54829
5,giseok1929 | giseok1929@daum.net,387,53682,2060,51622
7,smilekhm | smilekhm@gmail.com,235,52366,5165,47201
0,quantlab | quantlab@kakao.com,917,54107,10785,43322
...,...,...,...,...,...
25,Seoul_domestic_h | magrittelim@gmail.com,0,0,0,0
62,Trend_cardet_h | magrittelim@gmail.com,0,0,0,0
19,Street_hound | magrittelim@gmail.com,0,0,0,0
102,yellowin20212021 | yellowin20212021@gmail.com,0,0,0,0


In [131]:
df.iloc[6:].to_excel('test.xlsx')#.iloc[:10]

# 래플참여 현황

In [109]:
raffle_result = trans.filter(type=101).values('sender__boo__user__email', 'receiver__raffle__item__name', 'amount')

In [122]:
df = pd.DataFrame(raffle_result)
df.columns = ['유저','래플','참여']
df = df[~df['유저'].isin(memb_list)]

In [123]:
df.groupby(by=['래플','유저']).count()

참여
래플                   유저                       
KINTO TUMBLER 350ML  dydwk231@naver.com      1
                     fredasd0120@gmail.com   1
                     heetae002@naver.com     7
                     jueun0618@gmail.com     1
                     jylee118218@naver.com   1
...                                         ..
오징어게임 프론트맨 가면        london1920h@naver.com   1
                     ssopie@gmail.com        1
                     thfk3333@gmail.com      2
존버 대용량 철인3종 수영가방 메시백 kimjjmm@hanmail.net     1
                     liokjkr@nate.com        1

[117 rows x 1 columns]

In [114]:
df.groupby(by=['래플','유저']).count().to_excel('raffle_stat_20211025.xlsx')

In [124]:
df.groupby(by=['래플','유저']).count().loc['오징어게임 프론트맨 가면'].sum()

참여    11
dtype: int64